In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bion

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
import argparse
import time
import json
import csv

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from tensorflow.keras.utils import Progbar

import argparse
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs

from datetime import date
import os
import pandas as pd
import numpy as np
# from webdriver_manager.chrome import ChromeDriverManager
# from tensorflow.keras.utils import Progbar

today = date.today().strftime("%Y_%m_%d")
print("Today's date:", today)

# create debug folder
debug_path = "run_" + today

if os.path.exists(debug_path):
    print("\'{}\' is already EXISTED! --> REMOVE OLD DIR...".format(debug_path))
    # shutil.rmtree(debug_path)
else:
    os.mkdir(debug_path)
    print("\'{}\' is CREATED!".format(debug_path))


def _extract_html(bs_data):
    # Add to check
    with open(debug_path+'/bs.html', "w", encoding="utf-8") as file:
        file.write(str(bs_data.prettify()))

    data = []
    table = bs_data.find('table', attrs={'id': 'event-content'})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')

        cols = [ele.text.strip() for ele in cols]
        data.append(cols)
        # cols = [' '.join(ele.split()) for ele in cols]
        # data.append([ele for ele in cols if ele])
    return data


def _extract_header(bs_data):
    header = []
    table = bs_data.find('table', attrs={'id': 'event-content'})
    table_body = table.find('thead')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('th')
        cols = [ele.text.strip() for ele in cols]
        cols = [' '.join(ele.split()) for ele in cols]
        header.append([ele for ele in cols if ele])
    return header


def _extract_max_page(bs_data):
    page = bs_data.find_all(class_="m-r-xs")
    txt = page[0].find_all('span')[1]
    max_page = txt.text.strip()
    return int(max_page)


def extract(stock_code: str, paging: int, start_date='2006-01-01'):
    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
    option.add_argument('--headless')
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')
    
    # Pass the argument 1 to allow and 2 to block
    option.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 2,
        "profile.default_content_setting_values.images": 2,
        "profile.managed_default_content_settings.images": 2
    })

    # chromedriver should be in the same folder as file
    # browser = webdriver.Chrome(ChromeDriverManager().install())
    browser = webdriver.Chrome(executable_path="/usr/lib/chromium-browser/chromedriver", options=option)
    browser.get("https://finance.vietstock.vn/lich-su-kien.htm?page=1&from={}&to=2021-01-08&code={}".format(start_date, stock_code.upper()))
    time.sleep(2)
    df = pd.DataFrame()

    max_page = 10
    for _id in range(1, paging+1):
        print("Page ",_id)
        if _id > max_page:
            break
        if _id != 1:
            # browser.find_element_by_link_text(str(_id)).click()
            # browser.find_element_by_css_selector(".login-btn > span.btn-text").click()
            # browser.find_element_by_xpath(""]")
            browser.find_element_by_id("btn-page-next").click()
        # Throw your source into BeautifulSoup and start parsing!
        source_data = browser.page_source
        bs_data = bs(source_data, 'html.parser')
        max_page = _extract_max_page(bs_data)
        postBigDict = _extract_html(bs_data)
        columns_name = _extract_header(bs_data)

        df_page = pd.DataFrame(postBigDict, columns=columns_name[0])
        print(df_page)
        df = pd.concat([df, df_page])
        time.sleep(np.random.uniform(1, 3))
    df[df.columns[3]] = pd.to_datetime(df[df.columns[3]], format="%d/%m/%Y")
    df = df.sort_values(by=df.columns[3], ascending=False)
    df = df.drop_duplicates()
    print(df.info())
    print(df)
    df.to_csv(debug_path+'/'+stock_code+'.csv', index=False)
    browser.close()


extract(stock_code='VNM', paging=5, start_date='2004-01-01')

Today's date: 2021_01_18
'run_2021_01_18' is CREATED!
Page  1
   STT  ...              Loại Sự kiện
0    1  ...  Trả cổ tức bằng tiền mặt
1    2  ...  Trả cổ tức bằng tiền mặt
2    3  ...           Thưởng cổ phiếu
3    4  ...  Trả cổ tức bằng tiền mặt
4    5  ...  Trả cổ tức bằng tiền mặt
5    6  ...  Trả cổ tức bằng tiền mặt
6    7  ...  Trả cổ tức bằng tiền mặt
7    8  ...  Trả cổ tức bằng tiền mặt
8    9  ...  Trả cổ tức bằng tiền mặt
9   10  ...           Thưởng cổ phiếu
10  11  ...  Trả cổ tức bằng tiền mặt
11  12  ...  Trả cổ tức bằng tiền mặt
12  13  ...  Trả cổ tức bằng tiền mặt
13  14  ...  Trả cổ tức bằng tiền mặt
14  15  ...  Trả cổ tức bằng tiền mặt
15  16  ...           Thưởng cổ phiếu
16  17  ...  Trả cổ tức bằng tiền mặt
17  18  ...  Trả cổ tức bằng tiền mặt
18  19  ...           Thưởng cổ phiếu
19  20  ...  Trả cổ tức bằng tiền mặt

[20 rows x 8 columns]
Page  2
   STT  ...              Loại Sự kiện
0    1  ...  Trả cổ tức bằng tiền mặt
1    2  ...  Trả cổ tức bằng tiền